 # **Hugging Face**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Price_Predict_with_LLMs

/content/drive/MyDrive/Price_Predict_with_LLMs


In [3]:
!pip install transformers==4.43.3 accelerate torch bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import pandas as pd
import json
import re

In [5]:
from google.colab import userdata
token = userdata.get('hugging_face_key')

In [6]:
!huggingface-cli login --token {token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.int8
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [8]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.int8,
    num_return_sequences=1,
)

In [9]:
def get_response(prompt):
  response = text_generator(prompt)
  #print(response)
  return response[0]['generated_text']

In [10]:
crawl_data = pd.read_csv('canhochungcu_hanoi_v2.csv')
descriptions = crawl_data['description']

In [11]:
df = pd.DataFrame(columns=['address', 'type', 'price', 'area', 'price_per_m2'])

example = {
    "description": """Bán căn hộ chung cư Indochina Plaza - IPH Xuân Thủy.
                    Diện tích: 211m².
                    Thiết kế 4 phòng ngủ, 1 phòng khách, bếp, 3 phòng tắm.
                    Tầng cao đẹp, tầm nhìn Panorama, bao chọn thành phố.
                    Nội thất full đồ cao cấp, chủ nhà thiết kế lại sang đẹp, khách chỉ cần xách valy về ở.
                    Sẵn sổ đỏ, đặc biệt sẵn slot ô tô miễn phí dưới hầm.
                    Giá: 15.3 tỷ.""",
    "address": "Indochina Plaza - IPH Xuân Thủy",
    "type": "chung cư",
    "price": "15,3 tỉ",
    "area": "211 m2",
    "price_per_m2": "NaN"
}

system_message = f"""You are an expert in real estate.
                    You are an intelligent text extraction and conversion assistant. Your task is to extract structured information from the given text and convert it into a pure JSON format.
                    The JSON should contain only the structured data extracted from the text, with no additional commentary, explanations, or extraneous information.
                    1. **Address:** Extract the address, which can be a street or a district, and may include personal names.
                    2. **Type:** "căn hộ" or "subpenhouse".
                    3. **Price Calculation:**
                        - "tỷ" = 1,000,000,000 VND
                        - "tr" or "triệu" = 1,000,000 VND
                        72tr/m2 is amount_per_m2 and 1.5 tỷ is price.
                    4. **Area:** Extract the area in square meters.
                    5. **Price per m2** Extract the price per square meter.
                    If there is missing information, fill it as NaN.
                    Take a look at the following example: {example}
                    If there are multiple real estate properties, divide the information into several JSON segments and ensure that each output stays within the token limit.
                    Please ensure that all extracted data is converted to a pure JSON format. In cases where the data is not available or is in a foreign language, handle it appropriately.
                    Output only the JSON data with no additional text or explanations. The JSON should include address, type, price, and area fields."""


In [12]:
def process_description(description, system_message):
    user_message = description
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    output = get_response(messages)
    json_data_str = output[-1]['content']
    start_idx = json_data_str.find('{')
    end_idx = json_data_str.rfind('}')
    if start_idx == -1 or end_idx == -1:
        print("Không tìm thấy JSON hợp lệ trong phản hồi.")
    json_data_str = '[' + json_data_str[start_idx:end_idx+1] + ']'
    #print("Chuỗi JSON nhận được:", json_data_str)

    try:
        json_data = json.loads(json_data_str)
        return json_data
    except json.JSONDecodeError as e:
        print("Lỗi phân tích JSON:", e)
        return None


In [13]:
for description in descriptions[:10]:
  json_data = process_description(description, system_message)
  if json_data is not None:
    df = pd.concat([df, pd.DataFrame(json_data)], ignore_index=True)
print(df)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


                                              address             type  \
0                                   30T Nam Trung Yên           căn hộ   
1   Đại Lộ Thăng Long, phường Tây Mỗ, quận Nam Từ ...           căn hộ   
2                              The Beverly Ocean Park           căn hộ   
3                              The Beverly Ocean Park           căn hộ   
4                              The Beverly Ocean Park           căn hộ   
5                              The Beverly Ocean Park           căn hộ   
6                              The Beverly Ocean Park           căn hộ   
7   Imperia Sola Park, Vinhomes Smart City, phường...  căn hộ chung cư   
8   Imperia Sola Park, Vinhomes Smart City, phường...  căn hộ chung cư   
9   Imperia Sola Park, Vinhomes Smart City, phường...  căn hộ chung cư   
10  Imperia Sola Park, Vinhomes Smart City, phường...  căn hộ chung cư   
11  Imperia Sola Park, Vinhomes Smart City, phường...  căn hộ chung cư   
12  8 Nguyên Hồng, phường Láng Hạ, quậ

In [14]:
df.to_excel('output.xlsx', index=False)

In [ ]:
!nvidia-smi

Wed Aug  7 08:06:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0              28W /  70W |  15101MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--